In [2]:
from geopandas import GeoDataFrame
from pandas.io.json import json_normalize
from pymongo import MongoClient
import pymongo
import pandas as pd
import requests
import os
from bs4 import BeautifulSoup
import re
import math
import geopandas as gpd
from geopy.distance import distance
from geopy import distance
from shapely.geometry import Point
import matplotlib.pyplot as plt
from matplotlib.collections import PatchCollection
from dotenv import load_dotenv
load_dotenv()
%matplotlib inline
import folium
from geopy.geocoders import Nominatim 
from functions import *
from pprint import pprint 
import numpy as np

# Comencé con las compañías que ya han recaudado al menos 1M$, los datos los obtengo de una base de datos llamada companies desde mongodb compass

In [3]:
client = MongoClient("mongodb://localhost/companies")
db = client.get_database()

In [4]:
compMoney = db.companies.find({"total_money_raised":{"$regex":".*M.*"}},{'name':1,'total_money_raised':1,'offices':1,'_id':0}).sort("total_money_raised",-1)
df = pd.DataFrame(compMoney)

In [5]:
df = df.explode('offices')
dfOfficeData = df[["offices"]].apply(lambda r: r.offices, result_type="expand", axis=1)
cleanData = pd.concat([df,dfOfficeData], axis=1)

In [6]:
cleanData = cleanData.drop(columns=["offices", "description","address2" ,"state_code", "country_code", "zip_code"])
cleanData = cleanData.dropna()

In [7]:
companiesAtlanta = cleanData[cleanData['city'].str.contains("Atlanta")]
#display(companiesAtlanta)
len(companiesAtlanta)

19

In [8]:
def asGeoJSON(lat,lng):
    try:
        lat = float(lat)
        lng = float(lng)
        if not math.isnan(lat) and not math.isnan(lng):
            return {
                "type":"Point",
                "coordinates":[lng,lat]
            }
    except Exception:
        print("Invalid data")
        return None


companiesAtlanta["location"] = companiesAtlanta[["latitude","longitude"]].apply(lambda x:asGeoJSON(x.latitude,x.longitude), axis=1)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


# Hago ahora Web Scrapping para conseguir las coordenadas del aeropuerto de Atlanta

In [9]:
url = 'https://es.wikipedia.org/wiki/Aeropuerto_Internacional_Hartsfield-Jackson'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')

In [10]:
airportLat = soup.select('span.latitude')
airportLatitud = [a.text for a in airportLat]
airportLatitud = airportLatitud[1].split(',')
airportLatitud = float(airportLatitud[0])

In [11]:
airportLon = soup.select('span.longitude')
airportLongitud = float([a.text for a in airportLon][1])

In [12]:
aeropuerto = pd.DataFrame({'longitude': airportLongitud, 'latitude': airportLatitud}, index=[0])

In [13]:
aeropuerto["location"] = aeropuerto[["latitude","longitude"]].apply(lambda x:asGeoJSON(x.latitude,x.longitude), axis=1)

# Ahora continuo con los Starbucks, los datos los obtengo de una base de datos llamada Starbucks desde mongodb compass

In [14]:
client = MongoClient("mongodb://localhost/starbucks")
db = client.get_database()

In [15]:
starbucksAtlanta = db.starbucks.find({"$and":[{"City":{"$eq":"Atlanta"}},{"Brand":{"$eq":"Starbucks"}}]},{'_id':0})
starbucksAtlanta = pd.DataFrame(starbucksAtlanta)

In [16]:
starbAtlanta = starbucksAtlanta.drop(columns=["Ownership Type", "Store Number", "Store Name", "Postcode", "State/Province", "Country", "Phone Number", "Timezone"])
starbAtlanta = starbAtlanta.rename(columns={"Brand": "name", "Street Address": "address1", "City": "city", "Latitude":"latitude", "Longitude": "longitude"})
len(starbAtlanta)

59

In [17]:
starbAtlanta["location"] = starbAtlanta[["latitude","longitude"]].apply(lambda x:asGeoJSON(x.latitude,x.longitude), axis=1)

# A continuación obtengo los colegios mediante la api de foursquare

In [18]:
url = "https://api.foursquare.com/v2/venues/explore?ll=33.74,-84.38&query=college&client_id=YXKP2CA5QFGFCV0503ZQBXIDBQXZCDV5MV2FSYZLSDNIEHCT&client_secret=MEMSU1KAINCBNI5RV3DWUOC5A2TV1DFVUAFLONH4CY33G2NP&v=20200210"
results = requests.get(url).json()
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)

In [19]:
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
atlantaCollege = nearby_venues.loc[:, filtered_columns]
atlantaCollege = atlantaCollege.rename(columns={"venue.name": "name", "venue.categories": "categories", "venue.location.lat": "latitude", "venue.location.lng": "longitude"})

atlantaCollege = atlantaCollege.explode('categories')
college = atlantaCollege[["categories"]].apply(lambda r: r.categories, result_type="expand", axis=1)
colegios = pd.concat([atlantaCollege,college], axis=1)
colegios = colegios.drop(columns=["categories", "id", "pluralName", "shortName", "icon"])
colegios.columns = ['name', 'latitude', 'longitude', 'type', 'primary']
len(colegios)

30

In [20]:
colegios["location"] = colegios[["latitude","longitude"]].apply(lambda x:asGeoJSON(x.latitude,x.longitude), axis=1)

# Ahora voy a unir las companies, los starbucks, el aeropuerto y los colegios por location (lat+long)

In [21]:
result_location = pd.concat([companiesAtlanta, starbAtlanta, aeropuerto, colegios])
#result_location = result_location.drop(columns=["latitude","longitude"])

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


# Exporto el resultado obtenido como .json para trabajarlo en mongo compass (en este punto no logré hacer lo que quería que era ver el mapa desde mongo compass, no me aparece la opción de schema)

In [22]:
#result_location.to_json("result_final.json", orient="records")
#print(type(resultado_location))

In [23]:
#client = MongoClient("mongodb://localhost/")
#db = client.get_database()

# Convierto los puntos obtenidos a GeoPoints para trabajarlos con Carto como Geopandas

In [24]:
def asGeo(name):
    return gpd.GeoDataFrame(name, geometry=gpd.points_from_xy(name.longitude, name.latitude))
gCompaniesAtlanta = asGeo(companiesAtlanta)

In [25]:
gAeropuerto = asGeo(aeropuerto)

In [26]:
gStarbAtlanta = asGeo(starbAtlanta)

In [27]:
gColegios = asGeo(colegios)

# Unimos gcompanies, gaeropuerto, gstarbucks y gcolegios por geometry para Geopandas

In [28]:
resultado_geometry = pd.concat([gCompaniesAtlanta, gStarbAtlanta, gAeropuerto, gColegios])
resultado_geometry = resultado_geometry.drop(columns=["latitude","longitude"])

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


# Ya teniendo todo junto obtengo un mapa mediante Carto, en el paso anterior los uní pero ahora los trabajo por separado para poder dar colores a cada uno

In [128]:
crs = {'init': 'epsg:4326'}
gdf = GeoDataFrame(resultado_geometry, crs=crs)

In [129]:
Map([Layer(gAeropuerto, 'color: blue'), Layer(gStarbAtlanta, 'color: green'), Layer(gColegios, 'color: yellow'), Layer(gCompaniesAtlanta, 'color: red')], interactive=True)

# Distance y Geoquery del punto de Atlanta que deseo

In [29]:
def geocode(address):
    data = requests.get(f"https://geocode.xyz/{address}?json=1").json()
    return {
        "type":"Point",
        "coordinates":[float(data["longt"]),float(data["latt"])]
    }
cocaCola_atlanta = geocode("Coca Cola Atlanta")
cocaCola_atlanta

{'type': 'Point', 'coordinates': [-84.38191, 33.75443]}

In [30]:
puntoAtlanta = geocode("Wall Street Atlanta")
puntoAtlanta

{'type': 'Point', 'coordinates': [-84.38915, 33.75268]}

In [31]:
geoquery_cocaCola = withGeoQuery(cocaCola_atlanta)
pprint(geoquery_cocaCola)

{'location': {'$near': {'$geometry': {'coordinates': [-84.38191, 33.75443],
                                      'type': 'Point'},
                        '$maxDistance': 3000,
                        '$minDistance': 0}}}


In [32]:
geoquery_puntoAtlanta = withGeoQuery(puntoAtlanta)
pprint(geoquery_puntoAtlanta)

{'location': {'$near': {'$geometry': {'coordinates': [-84.38915, 33.75268],
                                      'type': 'Point'},
                        '$maxDistance': 3000,
                        '$minDistance': 0}}}


In [33]:
db = client.get_database("companies")
#list(db["resultado_final"].find({}))

In [34]:
cerca_cocaCola = list(db["resultado_final"].find(geoquery_cocaCola))
print(len(cerca_cocaCola))

45


In [35]:
punto_centro_Atlanta = list(db["resultado_final"].find(geoquery_puntoAtlanta))
print(len(punto_centro_Atlanta))

42


In [36]:
result = pd.DataFrame(cerca_cocaCola)
starbucks_totales_radio3km = result[result['name'].str.contains("Starbucks")]
print(len(starbucks_totales_radio3km))

13


In [37]:
result = pd.DataFrame(punto_centro_Atlanta)
starbucks_totales_radio3km = result[result['name'].str.contains("Starbucks")]
print(len(starbucks_totales_radio3km))

9


In [38]:
result = pd.DataFrame(cerca_cocaCola)
colegios_totales_radio3km = result[result.type.notnull()]
print(len(colegios_totales_radio3km))

28


In [39]:
result = pd.DataFrame(punto_centro_Atlanta)
colegios_totales_radio3km = result[result.type.notnull()]
print(len(colegios_totales_radio3km))

29


In [40]:
result = pd.DataFrame(cerca_cocaCola)
companies_totales_radio3km = result[result['total_money_raised'].str.contains("M", na = False)]
print(len(companies_totales_radio3km))

4


In [41]:
result = pd.DataFrame(punto_centro_Atlanta)
companies_totales_radio3km = result[result['total_money_raised'].str.contains("M", na = False)]
print(len(companies_totales_radio3km))

4


In [42]:
puntoAtlanta = (33.7772, -84.3901)
hartsfield_Jackson = (33.3812, -84.2541)
print(distance.distance(puntoAtlanta, hartsfield_Jackson).km)

45.70096668267049


In [43]:
cocaCola =  (33.75443,-84.38191)
hartsfield_Jackson = (33.3812, -84.2541)
print(distance.distance(hartsfield_Jackson, cocaCola).km)

43.06404143269257


In [44]:
# Obtuve los sitios cercanos a 2 puntos. Y como resultado pondría mi empresa en el punto del centro de Atlanta y no es
# porque haya dado 1 empresa más que el punto de Coca Cola Company, sería porque tendría cerca una opción de 
# colegio más y adicionalmente 1 empresa más que haya obtenido el total_money_raised que busqué inicialmente.
# Las distancias de ambos puntos con respecto al aeropuerto son bastante similares por lo tanto no influyen en la 
# toma de mi decisión.